# Plotting model fit results for all patients

Use m2c results

In [ ]:
import sys
sys.path.append('../')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from new_patient_model import build_pm_model_from_dataframes, extract_data_from_tables_new,\
        run_model, plot_data, plot_runs, calculate_errors, split_train_test,\
        plot_runs_area

from find_map import fit_model_map, pybobyqa_wrapper

from simplified_models import model_desc_m2c, build_pm_model_m2c, initialization_fn_m2c_2,\
          param_names_m2c, param_bounds_m2b, param_bounds_m2c, generate_dosing_component_m2


In [ ]:
# loading patient data
patient_ids = np.loadtxt('../id_samples_3_cycles.txt', dtype=int)
blood_counts = pd.read_excel('../patient_data_venex/Ven_blood_counts_16042023.xlsx', sheet_name='Blood_counts')
bm_blasts = pd.read_excel('../patient_data_venex/Ven_blood_counts_16042023.xlsx', sheet_name='Bone_marrow_blasts')
cycle_days = pd.read_excel('../patient_data_venex/Ven_blood_counts_16042023.xlsx', sheet_name='Cycle_days')

# loading patient information
patient_data = pd.read_csv('../patient_data_venex/ven_responses_052023.txt', sep='\t')
neut_blast_correlations = pd.read_csv('../neut_blast_treatment_correlations.csv', index_col=0)

In [ ]:
m2c_param_data = pd.read_csv('../systematic_comparison_results_simplified_models/m2c_mp_param_data.csv', index_col=0)
m2c_rmse_data = pd.read_csv('../systematic_comparison_results_simplified_models/m2c_mp_rmse_data.csv', index_col=0)


## Plot patients

In [ ]:
model_names = ['m2c']
model_params = [m2c_param_data]
model_rmses = [m2c_rmse_data]

#model_names = ['m4b_wbc_only', 'm2a', 'm2b', 'm2c', 'm2d', 'm2b_wbc_only']
#model_params = [m4b_wbc_only_param_data,
#                m2a_param_data, m2b_param_data, m2c_param_data, m2d_param_data, m2b_wbc_only_param_data,
#                ]
#model_rmses = [m4b_wbc_only_rmse_data,
#               m2a_rmse_data, m2b_rmse_data, m2c_rmse_data, m2d_rmse_data, m2b_wbc_only_rmse_data,
#               ]

In [ ]:
# TODO: calculate frac unexplained variance using the 1 - r^2 formula
from systematic_model_comparisons_multiprocessing import MODELS
from new_patient_model import calculate_errors, build_pm_model_from_dataframes
from tellurium_model_fitting import rmse, get_model_results_times

data_neut_values = {}
model_neut_values = {}

for model_name, param_table, rmse_table in zip(model_names, model_params, model_rmses):
    print()
    print(model_name)
    print()
    model_neut_values[model_name] = []
    data_neut_values[model_name] = []
    model_functions = MODELS[model_name]
    model_desc = model_functions['model_desc']
    build_pm_model = model_functions['build_pm_model']
    initialization_fn = model_functions['initialization_fn']
    param_names = model_functions['param_names']
    param_bounds = model_functions['param_bounds']
    dosing_component = None
    if 'dosing_component' in model_functions:
        dosing_component = model_functions['dosing_component']
    use_blasts = True
    if 'wbc_only' in model_functions:
        use_blasts = not model_functions['wbc_only']
    neut_correlations = []
    blast_correlations = []
    # MAPE - mean absolute percentage error
    mape_neuts = []
    mape_blasts = []
    for patient_id, rmse_val in rmse_table.leuk_train.items():
        try:
            param_vals = param_table.loc[patient_id].to_list()
        except:
            print(patient_id, 'params not found')
            neut_correlations.append(np.inf)
            blast_correlations.append(np.inf)
            continue
        cycle_info, leuk_table, blast_table = extract_data_from_tables_new(blood_counts,
                                                                             bm_blasts, cycle_days, patient_id, use_neut=True)
        te_model, pm_model = build_pm_model_from_dataframes(cycle_info, leuk_table, blast_table,
            use_neut=True, use_b0=True,
            model_desc=model_desc,
            build_model_function=build_pm_model,
            initialization=initialization_fn,
            params_to_fit=param_names,
            dosing_component_function=dosing_component,
            uniform_prior=True,
            use_initial_guess=True,
            use_blasts=use_blasts
            )
        print('Patient ' + str(patient_id))
        plot_data(None, cycle_info, leuk_table, blast_table, patient_id, save_fig=False, use_neut=True)
        if np.isinf(rmse_val):
            print('rmse is inf')
            neut_correlations.append(np.inf)
            blast_correlations.append(np.inf)
            mape_neuts.append(np.inf)
            continue
        try:
            neut_corr, blast_corr, results = calculate_errors(te_model, param_vals, cycle_info, leuk_table, blast_table,
                                                   params_to_fit=param_names, wbc_only=not use_blasts, use_neut=True,
                                                   initialization=initialization_fn, error_fn='corrcoef')
        except:
            neut_corr = np.inf
            blast_corr = np.inf
            results = None
        neut_correlations.append(neut_corr)
        blast_correlations.append(blast_corr)
        if results is not None:
            plot_data(results, cycle_info, leuk_table, blast_table, patient_id, save_fig=False, use_neut=True)
            results_points = get_model_results_times(results['Xwbc'], results['time'], leuk_table.lab_date.to_numpy())
            data_neut_values[model_name] += leuk_table.b_neut.to_list()
            model_neut_values[model_name] += results_points
            mape_neuts.append(rmse(results['Xwbc'], results['time'], leuk_table.b_neut.to_numpy(), leuk_table.lab_date.to_numpy(), 'mape'))
        else:
            mape_neuts.append(np.inf)
        print('neut_corr:', neut_corr, 'blast_corr:', blast_corr)